In [2]:
import boto3
import os

In [50]:
def create_key_pair():
    ec2_client = boto3.client("ec2", region_name="us-east-1")
    key_pair = ec2_client.create_key_pair(KeyName="lab4")
    private_key = key_pair["KeyMaterial"]
create_key_pair()

In [3]:
#create instance
def create_instance():
    ec2_client = boto3.client("ec2", region_name="us-east-1")
    instances = ec2_client.run_instances(
        ImageId="ami-08c40ec9ead489470",
        MinCount=1,
        MaxCount=1,
        InstanceType="t2.micro",
        KeyName="lab4"
    )

    print(instances["Instances"][0]["InstanceId"])
    
create_instance()

i-0d17caabb03880762


In [4]:
def get_public_ip(instance_id):
     ec2_client = boto3.client("ec2", region_name="us-east-1")
     reservations = ec2_client.describe_instances(InstanceIds=[instance_id]).get("Reservations")
     for reservation in reservations:
         for instance in reservation['Instances']:
             print(instance.get("PublicIpAddress"))


In [6]:
get_public_ip('i-0d17caabb03880762')

34.224.25.155


In [7]:
def get_running_instances():
     ec2_client = boto3.client("ec2", region_name="us-east-1")
     reservations = ec2_client.describe_instances(Filters=[
     {
     "Name": "instance-state-name",
     "Values": ["running"],
     },
     {
     "Name": "instance-type",
     "Values": ["t2.micro"]
     }
     ]).get("Reservations")
    
     for reservation in reservations:
         for instance in reservation["Instances"]:
             instance_id = instance["InstanceId"]
             instance_type = instance["InstanceType"]
             public_ip = instance["PublicIpAddress"]
             private_ip = instance["PrivateIpAddress"]
             print(f"{instance_id}, {instance_type}, {public_ip}, {private_ip}")

get_running_instances()

i-0d17caabb03880762, t2.micro, 34.224.25.155, 172.31.25.190
i-0ee5f6c714678d631, t2.micro, 3.80.156.175, 172.31.95.24


In [9]:
def stop_instance(instance_id):
     ec2_client = boto3.client("ec2", region_name="us-east-1")
     response = ec2_client.stop_instances(InstanceIds=[instance_id])
     print(response)
stop_instance('i-0d17caabb03880762')

{'StoppingInstances': [{'CurrentState': {'Code': 64, 'Name': 'stopping'}, 'InstanceId': 'i-0d17caabb03880762', 'PreviousState': {'Code': 16, 'Name': 'running'}}], 'ResponseMetadata': {'RequestId': '066ed728-744c-4154-9b94-49b59e55e1a7', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '066ed728-744c-4154-9b94-49b59e55e1a7', 'cache-control': 'no-cache, no-store', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'content-type': 'text/xml;charset=UTF-8', 'content-length': '579', 'date': 'Fri, 02 Dec 2022 17:39:37 GMT', 'server': 'AmazonEC2'}, 'RetryAttempts': 0}}


In [12]:
def terminate_instance(instance_id):
     ec2_client = boto3.client("ec2", region_name="us-east-1")
     response = ec2_client.terminate_instances(InstanceIds=[instance_id])
     print(response)
terminate_instance('i-09750ee4baa39fa2b')


{'TerminatingInstances': [{'CurrentState': {'Code': 48, 'Name': 'terminated'}, 'InstanceId': 'i-09750ee4baa39fa2b', 'PreviousState': {'Code': 80, 'Name': 'stopped'}}], 'ResponseMetadata': {'RequestId': 'f6318cd0-bc5a-4088-918e-49b3e37e3b01', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'f6318cd0-bc5a-4088-918e-49b3e37e3b01', 'cache-control': 'no-cache, no-store', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'vary': 'accept-encoding', 'content-type': 'text/xml;charset=UTF-8', 'transfer-encoding': 'chunked', 'date': 'Mon, 28 Nov 2022 12:13:31 GMT', 'server': 'AmazonEC2'}, 'RetryAttempts': 0}}


In [27]:
from boto.s3.connection import S3Connection

In [57]:
#S3
s3 = boto3.client('s3')
conn = S3Connection()
Bucket='lab4test'
if conn.lookup(Bucket) is not None:
     print('Something went wrong')
else:
    s3.create_bucket(Bucket='lab4test')

In [56]:
#bucket list
s3 = boto3.client('s3')
response = s3.list_buckets()
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f' {bucket["Name"]}')

Existing buckets:
 bucket-lab-2


In [42]:
# upload file

s3 = boto3.client('s3')

filename = 'test.txt'
bucket_name = 'lab4test'
s3.upload_file(filename, bucket_name, filename)

In [48]:
#read
import pandas as pd

In [54]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('lab4test') 
key = 'error.csv'      
obj = list(bucket.objects.filter(Prefix=key))
if len(obj) > 0:
    df = pd.read_csv(key)
    print(df.head())
else:
    print('Something went wrong')

Something went wrong


In [58]:
# delete non-empty bucket
bucket = boto3.resource('s3').Bucket('lab4test')
bucket.objects.all().delete()
bucket.delete()

{'ResponseMetadata': {'RequestId': '8W06TFT5RQP0KR5K',
  'HostId': 'k9Wrd00hA/j5gmrpuqa+KKlbf9EVR46dx+ELjzbiih0kQqf39INeJmvVLSO8vOP/qn6eo7PljoQ=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'k9Wrd00hA/j5gmrpuqa+KKlbf9EVR46dx+ELjzbiih0kQqf39INeJmvVLSO8vOP/qn6eo7PljoQ=',
   'x-amz-request-id': '8W06TFT5RQP0KR5K',
   'date': 'Mon, 28 Nov 2022 13:43:16 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}